# Chapter 16 - Natural Language Processing: Applications

Given pretrained text representations, we will explore how sentiment analysis and natural language inference can be accomplished in practice.

![](../imgs/ch16/nlp-map-app.svg)

## 16.1. Sentiment Analysis and the Dataset